In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import google.auth
import pandas as pd
import os
import pygsheets
import openpyxl

### sheet link to รับสินค้าเข้า DHL
database_sheet_link = "1IuaS4S45dgBRpUb3lqJf3VxESDUMIB3hgBRhHUmRrf0"


In [2]:
#define scope and then authorize JSON key
scope_app = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.cp,/auth/drive']

get_json_key = 'itecdatabase-b286a-2c2408acc426.json'

if get_json_key in os.listdir():
    spreadsheet = pygsheets.authorize(service_account_file=get_json_key)
else:
    try:
        os.chdir("D:\Workstuff\my-work-python-script\Database_in_pandas\data_to_sheet")
        spreadsheet = pygsheets.authorize(service_account_file=get_json_key)
    except:
        print("missing JSON API key")

#open sheet
sheet = spreadsheet.open("รับสินค้าเข้า (DHL&Zone)")

In [3]:
getdata = sheet.worksheet('id', value=1591433816)

In [8]:

days = 1 # 1 = yesterday, 2 = day before yesterday etc etc you know it.
formula_33 = f'''=QUERY('รับ'!B:N,"SELECT B,F,G,J WHERE B CONTAINS date '"&TEXT(DATEVALUE(TODAY()-{days}),"yyyy-mm-dd")&"' AND H MATCHES '33 Service Headoffice' AND J MATCHES '.*Zone.*|.*Pantip.*' ",1)'''
formula_33ins = f'''=QUERY('รับ'!B:N,"SELECT B,F,G,J WHERE B CONTAINS date '"&TEXT(DATEVALUE(TODAY()-{days}),"yyyy-mm-dd")&"' AND H MATCHES '33 Insure' AND J MATCHES '.*Zone.*|.*Pantip.*' ",1)'''
formula_49ins = f'''=QUERY('รับ'!B:N,"SELECT B,F,G,J WHERE B CONTAINS date '"&TEXT(DATEVALUE(TODAY()-{days}),"yyyy-mm-dd")&"' AND H MATCHES '49 Trade In' AND J MATCHES '.*Zone.*|.*Pantip.*' ",1)'''
formula_49 = f'''=QUERY('รับ'!B:N,"SELECT B,F,G,J WHERE B CONTAINS date '"&TEXT(DATEVALUE(TODAY()-{days}),"yyyy-mm-dd")&"' AND H MATCHES '49 Return' AND J MATCHES '.*Zone.*|.*Pantip.*' ",1)'''

# 29 cols
#stockout 33
getdata.update_value('A1', formula_33)
stockout_33 = pd.DataFrame(getdata.get_values(('A2'), (getdata.rows,getdata.cols-25)))

#stockout33_ins
getdata.update_value('F1', formula_33ins)
stockout_33ins = pd.DataFrame(getdata.get_values('F2', (getdata.rows,getdata.cols-20)))

#stockout49_ins
getdata.update_value('K1', formula_49ins)
stockout_49ins = pd.DataFrame(getdata.get_values('K2', (getdata.rows,getdata.cols-15)))

#stockout49
getdata.update_value('P1', formula_49)
stockout_49 = pd.DataFrame(getdata.get_values('P2', (getdata.rows,getdata.cols-10)))


<H1>REPLACE</h1>

In [5]:
excel_dir = os.chdir(r"D:\Workstuff\my-work-python-script\Docref")

excel_file = "bitly+ready.xlsx"

if excel_file in os.listdir():
    with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as file:
        stockout_33.to_excel(file, sheet_name="33_BKK", index=False, header=False)
        stockout_33ins.to_excel(file, sheet_name="33_INS_BKK", index=False, header=False)
        stockout_49ins.to_excel(file, sheet_name="49TradeBKK", index=False, header=False)
        stockout_49.to_excel(file, sheet_name="49RETURN+747+37-BKK", index=False, header=False)

<H1>OVERLAY</h1>


In [9]:
excel_dir = os.chdir(r"D:\Workstuff\my-work-python-script\Docref")

excel_file = "bitly+ready.xlsx"

if excel_file in os.listdir():
    with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="overlay") as file:
        stockout_33.to_excel(file, sheet_name="33_BKK", index=False, header=False, startrow=file.sheets["33_BKK"].max_row)
        stockout_33ins.to_excel(file, sheet_name="33_INS_BKK", index=False, header=False,startrow=file.sheets["33_INS_BKK"].max_row)
        stockout_49ins.to_excel(file, sheet_name="49TradeBKK", index=False, header=False, startrow=file.sheets["49TradeBKK"].max_row)
        stockout_49.to_excel(file, sheet_name="49RETURN+747+37-BKK", index=False, header=False, startrow=file.sheets["49RETURN+747+37-BKK"].max_row)